In [1]:
import torch
import cv2 as cv
import os
from PIL import Image
from tqdm import tqdm
from torchvision import transforms
import torchvision.models as models
import matplotlib.pyplot as plt
import json
import torch
import warnings
import re
import numpy as np

# /!\ With all the videos this notebook run over 2 days

See results for the first video there : https://drive.google.com/drive/folders/13dLyIqrqQul0PRI5OHjJ4nDpeSrzBlf9?usp=sharing

It runs AlexNet, Yolo and Inception. We use the key frame found with clustering as input. Then the output is a dictionary. 

{video_1 : {scene_1 : {cluster_1 : {concept1 : n_occurrence_1, concept2 : n_occurrence_2}
                                    
                      cluster_2 : {...},
                      
                      },
                      
           scene_2 :  {...}
           
 video_2 : {...} }
 
 We use a threshold of 0.66 for keeping a concept or object and n_occurrence represente the number of time where the algorithm find this concept/object on a frame in a scene

In [2]:
# this path chose where you want to load yolo
path_root = "C:\\Users\\baptc\\Documents\\computer_vision\\project_videos"

# This path points to the foler with the result of the keyframe extraction.
# It uses the most representative image of a scene (there can be several) of a cluster as input.
path_cluster = "C:\\Users\\baptc\\Documents\\computer_vision\\project_videos\\video_key_frame"

load yolo in the root of your project

In [3]:
os.chdir(path_root)
model = torch.hub.load('ultralytics/yolov5', 'yolov5x6')

Using cache found in C:\Users\baptc/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-6-9 Python-3.8.13 torch-1.11.0+cpu CPU

Fusing layers... 
YOLOv5x6 summary: 574 layers, 140730220 parameters, 0 gradients
Adding AutoShape... 


Verified the current workspace is in the folder with video key frame clustering

In [4]:
os.chdir(path_cluster)
os.getcwd()

'C:\\Users\\baptc\\Documents\\computer_vision\\project_videos\\video_key_frame'

In [5]:
# this is the path where you want to save the results
path_json = "C:\\Users\\baptc\\Documents\\computer_vision\\project_videos\\save_json"
def save_dct_as_json(dct,name,path):
    os.chdir(path)
    with open(f'{name}.json', 'w') as fp:
        json.dump(dct, fp)

## YOLO

In [6]:
with_ = False
if with_:
    os.chdir(path_cluster)
    dct_yolo = {}
    for path_video in tqdm(os.listdir()):
        dct_yolo[path_video]={}
        for path_scene in os.listdir(path_video):
            dct_yolo[path_video][path_scene]={}
            for path_key_frame in os.listdir(path_video+"\\"+path_scene):
                im = cv.imread(path_video+"\\"+path_scene+"\\"+path_key_frame)
                results = model(im)
                plen = len(results.pandas().xyxy[0])
                if plen > 0:
                    idx = 0
                    while idx < plen:
                        conf = results.pandas().xyxy[0]['confidence'][idx]
                        classnum = results.pandas().xyxy[0]['class'][idx]
                        name = results.pandas().xyxy[0]['name'][idx]
                        #print(f'{name},{classnum},{conf}')
                        if conf>0.66:
                            if name not in dct_yolo[path_video][path_scene].keys():
                                dct_yolo[path_video][path_scene][name]=1
                            else:
                                dct_yolo[path_video][path_scene][name]+=1
                        idx = idx + 1
    save_dct_as_json(dct_yolo,"yolo", path_json)

## AlexNet

In [7]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#https://pytorch.org/hub/pytorch_vision_alexnet/
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
model.eval().to(device)

Using cache found in C:\Users\baptc/.cache\torch\hub\pytorch_vision_v0.10.0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

### load this txt file and the remove the first 4th line (start with : 0, tench)

In [8]:
os.chdir(path_root)
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

In [9]:
with_ = False
if with_:
    os.chdir("C:\\Users\\baptc\\Documents\\computer_vision\\project_videos\\video_key_frame")
    dct_alexNet = {}
    for path_video in tqdm(os.listdir()):
        dct_alexNet[path_video]={}
        for path_scene in os.listdir(path_video):
            dct_alexNet[path_video][path_scene]={}
            for path_key_frame in os.listdir(path_video+"\\"+path_scene):
                input_image = Image.open(path_video+"\\"+path_scene+"\\"+path_key_frame)

                preprocess = transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ])
                input_tensor = preprocess(input_image)
                input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
                input_batch = input_batch.to(device)

                with torch.no_grad():
                    output = model(input_batch)

                probabilities = torch.nn.functional.softmax(output[0], dim=0)


                # Show top categories per image
                top_prob, top_catid = torch.topk(probabilities, 1)
                for i in range(top_prob.size(0)):
                    if top_prob[i].item()>0.66:
                        #display(input_image)
                        name = categories[top_catid[i]].split(",")[1]
                        name = re.sub("\s+","",name)
                        name = re.sub("_"," ",name)
                        name = re.sub("-"," ",name)
                        if name not in dct_alexNet[path_video][path_scene].keys():
                            dct_alexNet[path_video][path_scene][name]=1
                        else:
                            dct_alexNet[path_video][path_scene][name]+=1
    save_dct_as_json(dct_alexNet, "alexNet", path_json)

## Inception

In [10]:
os.chdir(path_cluster)
warnings.filterwarnings('ignore')

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=True)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')
model.eval().to(device) #evaluation behavior


Using cache found in C:\Users\baptc/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub
Using cache found in C:\Users\baptc/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


EfficientNet(
  (stem): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (activation): SiLU(inplace=True)
  )
  (layers): Sequential(
    (0): Sequential(
      (block0): MBConvBlock(
        (depsep): Sequential(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (se): SequentialSqueezeAndExcitation(
          (squeeze): Linear(in_features=32, out_features=8, bias=True)
          (expand): Linear(in_features=8, out_features=32, bias=True)
          (activation): SiLU(inplace=True)
          (sigmoid): Sigmoid()
          (mul_a_quantizer): Identity()
          (mul_b_quantizer): Identity()
        )
      

In [11]:
with_ = False
if with_:
    dct_inception = {}
    for path_video in tqdm(os.listdir()):
        dct_inception[path_video]={}
        for path_scene in os.listdir(path_video):
            dct_inception[path_video][path_scene]={}
            for path_key_frame in os.listdir(path_video+"\\"+path_scene):
                batch = torch.cat(
                [utils.prepare_input_from_uri(f'C:\\Users\\baptc\\Documents\\computer_vision\\project_videos\\video_key_frame\\{path_video}\\{path_scene}\\{path_key_frame}')]
                ).to(device)

                with torch.no_grad():
                    output = torch.nn.functional.softmax(model(batch), dim=1)
                prob = np.max(output.numpy())*100
                if prob>66:
                    name = re.sub("\s+","",categories[np.argmax(output)].split(',')[1])
                    name = re.sub("_"," ",name)
                    name = re.sub("-"," ",name)
                    if name not in dct_inception[path_video][path_scene].keys():
                        dct_inception[path_video][path_scene][name]=1
                    else:
                        dct_inception[path_video][path_scene][name]+=1
    save_dct_as_json(dct_inception, "inception", path_json)
